In [7]:
import pandas as pd

In [ ]:
train_rollout_dir = "~/persistent/rollouts/deepseek_r1_7b_gh_patches_2k_fixed_reward/train"
df = pd.read_json("~/persistent/rollouts/deepseek_r1_7b_gh_patches_2k_fixed_reward/train/77.jsonl", lines=True)
df.head()

In [ ]:
import matplotlib.pyplot as plt
import os
import glob
import numpy as np

# Get all JSONL files in the directory
jsonl_files = sorted(glob.glob(os.path.join(os.path.expanduser(train_rollout_dir), "*.jsonl")))

# Filter to every 10th file
every_10th_files = jsonl_files[::10]

# Create subplots
fig, axes = plt.subplots(len(every_10th_files), 1, figsize=(12, 3*len(every_10th_files)))
if len(every_10th_files) == 1:
    axes = [axes]

# Plot histogram for each file
for i, file_path in enumerate(every_10th_files):
    # Read the JSONL file
    df = pd.read_json(file_path, lines=True)
    
    # Calculate mean and median
    mean_score = df['score'].mean()
    median_score = df['score'].median()
    
    # Create histogram
    axes[i].hist(df['score'], bins=30, alpha=0.7, edgecolor='black')
    
    # Add vertical lines for mean and median
    axes[i].axvline(mean_score, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_score:.3f}')
    axes[i].axvline(median_score, color='green', linestyle='-', linewidth=2, label=f'Median: {median_score:.3f}')
    
    axes[i].set_title(f'Score Distribution - {os.path.basename(file_path)}')
    axes[i].set_xlabel('Score')
    axes[i].set_ylabel('Frequency')
    axes[i].grid(True, alpha=0.3)
    axes[i].legend()

plt.tight_layout()
plt.show()

# Print the files being plotted
print(f"Plotted {len(every_10th_files)} files:")
for file_path in every_10th_files:
    print(f"  {os.path.basename(file_path)}")


In [ ]:
# Count perfect scores (score = 1.0) in each JSONL file
print("Perfect scores (score = 1.0) per JSONL file:")
print("=" * 50)

for file_path in jsonl_files:
    df = pd.read_json(file_path, lines=True)
    perfect_scores = (df['score'] == 1.0).sum()
    total_samples = len(df)
    percentage = (perfect_scores / total_samples) * 100 if total_samples > 0 else 0
    
    print(f"{os.path.basename(file_path)}:")
    print(f"  Perfect scores: {perfect_scores}/{total_samples} ({percentage:.2f}%)")
    print()

# Summary statistics
print("Summary:")
print("=" * 20)
all_perfect_scores = []
all_total_samples = []

for file_path in jsonl_files:
    df = pd.read_json(file_path, lines=True)
    perfect_scores = (df['score'] == 1.0).sum()
    total_samples = len(df)
    all_perfect_scores.append(perfect_scores)
    all_total_samples.append(total_samples)

total_perfect = sum(all_perfect_scores)
total_samples = sum(all_total_samples)
overall_percentage = (total_perfect / total_samples) * 100 if total_samples > 0 else 0

print(f"Total perfect scores across all files: {total_perfect}/{total_samples} ({overall_percentage:.2f}%)")
print(f"Average perfect scores per file: {np.mean(all_perfect_scores):.2f}")
print(f"Min perfect scores in a file: {min(all_perfect_scores)}")
print(f"Max perfect scores in a file: {max(all_perfect_scores)}")


In [ ]:
# Create a dataframe with all rollouts that have perfect scores (score = 1.0)
perfect_rollouts = []

for file_path in jsonl_files:
    df = pd.read_json(file_path, lines=True)
    # Filter for perfect scores
    perfect_df = df[df['score'] == 1.0].copy()
    # Add source file information
    perfect_df['source_file'] = os.path.basename(file_path)
    perfect_rollouts.append(perfect_df)

# Combine all perfect rollouts into one dataframe
if perfect_rollouts:
    all_perfect_rollouts = pd.concat(perfect_rollouts, ignore_index=True)
    print(f"Found {len(all_perfect_rollouts)} rollouts with perfect scores")
    print(f"Shape of perfect rollouts dataframe: {all_perfect_rollouts.shape}")
    print("\nFirst few perfect rollouts:")
    print(all_perfect_rollouts.head())
else:
    print("No perfect rollouts found")
    all_perfect_rollouts = pd.DataFrame()


In [ ]:
print(all_perfect_rollouts["output"][30])